In [1]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoTweetStore
import pymongo

In [2]:
loader = TweetLoader()
loader.extract_tweets(count=10)

In [3]:
len(loader.get_loaded_tweets_as_json())

10

In [4]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

Fox News
Bloomberg
Forbes
Bloomberg
Bloomberg
Bloomberg
The Penny Hoarder
Forbes
The Wall Street Journal
Fox News


In [5]:
store = MongoTweetStore("twitter_dashboard_db", "home_timeline")

In [6]:
store.collection.create_index([("id", pymongo.ASCENDING)], name="tweet_id", unique=True, default_language="english")

'tweet_id'

In [9]:
store.save_tweets_to_db(loader.tweets)

1. Fox News
Duplicate tweet id: 1606688688167059458
2. Forbes
Duplicate tweet id: 1606688699743453186
3. The Wall Street Journal
Duplicate tweet id: 1606688698506133506
4. The Penny Hoarder
Duplicate tweet id: 1606688862960599040
5. Bloomberg
Duplicate tweet id: 1606688911752941568
6. Bloomberg
Duplicate tweet id: 1606689670368305154
7. Bloomberg
Duplicate tweet id: 1606690673104756737
8. Forbes
Duplicate tweet id: 1606691188446224385
9. Fox News
Duplicate tweet id: 1606691191445151744
10. Bloomberg
Duplicate tweet id: 1606691189448753154


In [8]:
# data = store.load_data()
data = store.load_flat_data()
len(data)

30

In [ ]:
data = store.load_data(latest=True)

for user_data in data:
    print(user_data["user"])
    print(len(user_data["tweets"]))
    for tweet in user_data["tweets"]:
        print(tweet["created_at"])

In [28]:
def print_documents(cursor):
    for document in cursor:
        print(document)
        # print(document["user"]["screen_name"])
        # print(document["id"])
        # print(document["text"])

# store.collection.aggregate(["$match": {"$in": ["WSJ"]}])
data = store.collection.find({"user.screen_name": {"$in": ["WSJ", "business"]}})
print_documents(data)


{'_id': ObjectId('63a6ffc7a6d2371ecb53b05a'), 'created_at': 'Sat Dec 24 13:30:12 +0000 2022', 'id': 1606643406385696770, 'id_str': '1606643406385696770', 'text': 'Covid-19’s rapid spread in China might help answer a question much of the world has been asking for a year: Is Omic… https://t.co/0qMpg6H8tS', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/0qMpg6H8tS', 'expanded_url': 'https://twitter.com/i/web/status/1606643406385696770', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3108351, 'id_str': '3108351', 'name': 'The Wall Street Journal', 'screen_name': 'WSJ', 'location': 'New York, NY', 'description': 'Sign up for our newsletters and aler

In [128]:
users = ["WSJ", "business", "NASA"]
n_tweets = 1000
n_user_tweets = 1000
num = 2
latest = False
pipeline = [
    # Filter only requested users
    # {"$match": {"user.screen_name": {"$in": users}}},
    # Group tweets by user
    {
        "$group": {
            "_id": "$user.screen_name",
            "docs": {"$push": "$$ROOT"}
            },
    },
]

data = store.collection.aggregate(pipeline)
# print_documents(data)

# Order alphabetically by name (Mongo doesn't return in specific order)
data = sorted(data, key=lambda x: x["_id"].lower())

tweets_count = 0
for group in data:
    if tweets_count >= n_tweets:
        break

    tweets = group["docs"]
    print(group["_id"])

    if len(tweets) > n_tweets - tweets_count:
        tweets = tweets[: n_tweets - tweets_count]

    if n_user_tweets:
        tweets = tweets[:n_user_tweets]
        # print(group["docs"])
        # print(len(group["docs"]))
        print(len(tweets))

    tweets_count += len(tweets)

business
11
celtics
2
FoodNetwork
1
Forbes
5
FoxNews
5
NASA
1
thepennyhoarder
1
WSJ
4


In [88]:
x = 1
y = 2
print(x,y)
x, y = y, x

print(x,y)

1 2
2 1
